In [7]:
print("hi all!")

hi all!


In [8]:
#Google Sheet Link: https://docs.google.com/spreadsheets/d/1RiU9c3YihCsAqh18C25WMfTxkOiAf-nXhxAWtEaKLA0/edit#gid=0
import pandas as pd
import numpy as np
import matplotlib as mpl

sheet_id = '1RiU9c3YihCsAqh18C25WMfTxkOiAf-nXhxAWtEaKLA0'
xls = pd.ExcelFile(f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=xlsx")

arrivals = pd.read_excel(xls, 'Arrivals', header = 0)
arrivals = arrivals.drop(['blank', 'Communication Type/Reason for Speaking.1'], axis=1)
departures = pd.read_excel(xls, 'Departures', header = 0)
departures = departures.drop(['Airport', 'Communication Type/Reason for Speaking.1'], axis=1)

,IATA,Flight No.,Tail No.,ICAO,Airplane Type,From - Airport,To - Airport,Departure Time,Arrival Time,Location on Airport,Runway/Takeway No.,Time of Communication,Tower Frequency,Who's Speaking?,Communication Type/Reason for Speaking,Message/Communication
0,AAL,462.0,N578UW,A76DF8,2014 AIRBUS A-321\nAMERICAN AIRLINES INC,PHX,ATL,5:25 PM MST,8:51 PM MST,Terminal 4; Gate A13,25R,5:26:01 PM MST,119.75,Pilot,Greeting,evening ground america 462 alpha 13 ready for ...
1,AAL,462.0,N578UW,A76DF8,2014 AIRBUS A-321\nAMERICAN AIRLINES INC,PHX,ATL,5:25 PM MST,8:51 PM MST,Terminal 4; Gate A13,25R,5:26:05 PM MST,119.75,ATC,Direction from ATC,american 462 thank you alpha 13 point lima app...
2,AAL,462.0,N578UW,A76DF8,2014 AIRBUS A-321\nAMERICAN AIRLINES INC,PHX,ATL,5:25 PM MST,8:51 PM MST,Terminal 4; Gate A13,25R,5:26:07 PM MST,119.75,Pilot,Response to ATC,we'll get to alpha and push until sub lima
3,AAL,462.0,N578UW,A76DF8,2014 AIRBUS A-321\nAMERICAN AIRLINES INC,PHX,ATL,5:25 PM MST,8:51 PM MST,C10,25R,5:30:54 PM MST,119.75,ATC,Direction from ATC,tac (taxi?) 462 to taxi from lima with alpha
4,AAL,462.0,N578UW,A76DF8,2014 AIRBUS A-321\nAMERICAN AIRLINES INC,PHX,ATL,5:25 PM MST,8:51 PM MST,C10,25R,5:30:58 PM MST,119.75,Pilot,Response to ATC,Standby
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,AAL,2664.0,N753NK,NaN,B738,PHX,TUS,6:19 PM MST,6:42 PM MST,NaN,NaN,6:22:29 PM MST,119.20,ATC,Direction to Climb,2664 phx depature radar contanct climb and mai...
71,AAL,2664.0,N753NK,NaN,B738,PHX,TUS,6:19 PM MST,6:42 PM MST,NaN,NaN,6:22:30 PM MST,119.20,Pilot,Noise,unintelligible
72,AAL,2664.0,N753NK,NaN,B738,PHX,TUS,6:19 PM MST,6:42 PM MST,NaN,NaN,6:23:48 PM MST,119.20,ATC,Direction to Climb,2664 turn right heading 250 vector for the climb
73,AAL,2664.0,N753NK,NaN,B738,PHX,TUS,6:19 PM MST,6:42 PM MST,NaN,NaN,6:23:52 PM MST,119.20,Pilot,Response to ATC,heading 250 marker 2664
